In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_ML\FeaturesLocal\FeaturesST"

## <span style="color:yellow"> **Preparamos los Sets de Training** </span> 

In [3]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_ML\Training Sets"

In [4]:
clk_trn = pd.read_csv( loc_ts + "\\clk_18_20.csv" ); 
clk_tst = pd.read_csv( loc_ts + "\\clk_21_23.csv" );

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [8]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_ML\Labels"

In [9]:
lb_auc_trn = pd.read_csv( loc_lb + "\\label_auc_21_23.csv" );
lb_auc_tst = pd.read_csv( loc_lb + "\\label_auc_24_26.csv" ); 

In [10]:
rh_trn = lb_auc_trn[ ["ref_hash"] ]
rh_tst = lb_auc_tst[ ["ref_hash"] ]

tg_trn = lb_auc_trn[ ["21_23_st"] ]
tg_tst = lb_auc_tst[ ["24_26_st"] ]

## <span style="color:yellow"> **=================================================================================================** </span> 

## <span style="color:green"> **Cantidad de Clicks en la ventana anterior** </span> 

In [ ]:
cant_clk_trn = rh_trn.copy()
cant_clk_tst = rh_tst.copy()

cant_clk_trn = cant_clk_trn.merge( clk_trn["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_clk_tst = cant_clk_tst.merge( clk_tst["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_clk_trn.fillna( 0, inplace = True )
cant_clk_tst.fillna( 0, inplace = True )

cant_clk_trn.to_csv( loc_ftr + "\\cant_clk_trn.csv", index = False )
cant_clk_tst.to_csv( loc_ftr + "\\cant_clk_tst.csv", index = False )

## <span style="color:green"> **Promedio del TimeToClick es un anuncio** </span> 

In [9]:
ttc_mean_trn = rh_trn.copy()
ttc_mean_tst = rh_tst.copy()

ttc_by_rh_trn = clk_trn[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_trn = ttc_by_rh_trn.reset_index()
ttc_by_rh_tst = clk_tst[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_tst = ttc_by_rh_tst.reset_index()

ttc_mean_trn = ttc_mean_trn.merge( ttc_by_rh_trn, how = "left", on = "ref_hash" )
ttc_mean_tst = ttc_mean_tst.merge( ttc_by_rh_tst, how = "left", on = "ref_hash" )

Casi todos los valores nulos!



## <span style="color:green"> **Tiempo hasta el primer click** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [10]:
frst_clk_trn = rh_trn.copy()
frst_clk_tst = rh_tst.copy()

first_click_rh_auc_trn = clk_trn[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_click_rh_auc_tst = clk_tst[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_click_rh_auc_trn["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_click_rh_auc_tst["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_click_rh_auc_trn.drop( "date", axis = 1, inplace = True )
first_click_rh_auc_tst.drop( "date", axis = 1, inplace = True )

frst_clk_trn = frst_clk_trn.merge( first_click_rh_auc_trn, how = "left", on = "ref_hash" )
frst_clk_tst = frst_clk_tst.merge( first_click_rh_auc_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_clk_trn.fillna( 3 * 24 * 3600, inplace = True )
frst_clk_tst.fillna( 3 * 24 * 3600, inplace = True )

frst_clk_trn.to_csv( loc_ftr + "\\frst_clk_trn.csv", index = False )
frst_clk_tst.to_csv( loc_ftr + "\\frst_clk_tst.csv", index = False )

## <span style="color:green"> **Recibio clicks entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [11]:
clk_trn['clk_16_20'] = ( pd.to_datetime( clk_trn['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_trn['date'] ).dt.hour < 21) 
clk_tst['clk_16_20'] = ( pd.to_datetime( clk_tst['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_tst['date'] ).dt.hour < 21)

clk_after_trn = rh_trn.copy()
clk_after_tst = rh_tst.copy()

hour_mode_trn = clk_trn.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_tst = clk_tst.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_trn = clk_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
clk_after_tst = clk_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

clk_after_trn["clk_16_20"] = (clk_after_trn["clk_16_20"] > 1).astype('int8')
clk_after_tst["clk_16_20"] = (clk_after_tst["clk_16_20"] > 1).astype('int8')

clk_after_trn.to_csv( loc_ftr + "\\clk_aftr_trn.csv", index = False )
clk_after_tst.to_csv( loc_ftr + "\\clk_aftr_tst.csv", index = False )

del clk_trn['clk_16_20']
del clk_tst['clk_16_20']

## <span style="color:green"> **Recibio clikcs entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [12]:
clk_trn['clk_21_3'] = ( pd.to_datetime( clk_trn['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_trn['date'] ).dt.hour > 20)
clk_tst['clk_21_3'] = ( pd.to_datetime( clk_tst['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_tst['date'] ).dt.hour > 20)

clk_night_trn = rh_trn.copy()
clk_night_tst = rh_tst.copy()

hour_mode_trn = clk_trn.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_tst = clk_tst.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_trn = clk_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
clk_night_tst = clk_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

clk_night_trn["clk_21_3"] =  (clk_night_trn["clk_21_3"] > 1).astype('int8')
clk_night_tst["clk_21_3"] =  (clk_night_tst["clk_21_3"] > 1).astype('int8')

clk_night_trn.to_csv( loc_ftr + "\\clk_ngth_trn.csv", index = False )
clk_night_tst.to_csv( loc_ftr + "\\clk_ngth_tst.csv", index = False )

del clk_trn['clk_21_3']
del clk_tst['clk_21_3']

## <span style="color:green"> **Recibio clicks entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [ ]:
clk_trn['clk_11_15'] = ( pd.to_datetime( clk_trn['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_trn['date'] ).dt.hour < 16) 
clk_tst['clk_11_15'] = ( pd.to_datetime( clk_tst['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_tst['date'] ).dt.hour < 16)

clk_midday_trn = rh_trn.copy()
clk_midday_tst = rh_tst.copy()

hour_mode_trn = clk_trn.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_tst = clk_tst.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_trn = clk_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
clk_midday_tst = clk_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

clk_midday_trn["clk_11_15"] =  (clk_midday_trn["clk_11_15"] > 1).astype('int8')
clk_midday_tst["clk_11_15"] =  (clk_midday_tst["clk_11_15"] > 1).astype('int8')

clk_midday_trn.to_csv( loc_ftr + "\\clk_mday_trn.csv", index = False )
clk_midday_tst.to_csv( loc_ftr + "\\clk_mday_tst.csv", index = False )

del clk_trn['clk_11_15']
del clk_tst['clk_11_15']

## <span style="color:green"> **Recibio clicks entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [ ]:
clk_trn['clk_4_10'] = ( pd.to_datetime( clk_trn['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_trn['date'] ).dt.hour < 11) 
clk_tst['clk_4_10'] = ( pd.to_datetime( clk_tst['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_tst['date'] ).dt.hour < 11)

clk_morn_trn = rh_trn.copy()
clk_morn_tst = rh_tst.copy()

hour_mode_trn = clk_trn.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_tst = clk_tst.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_trn = clk_morn_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
clk_morn_tst = clk_morn_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

clk_morn_trn["ins_4_10"] =  (clk_morn_trn["clk_4_10"] > 1).astype('int8')
clk_morn_tst["ins_4_10"] =  (clk_morn_tst["clk_4_10"] > 1).astype('int8')

clk_morn_trn.to_csv( loc_ftr + "\\clk_morn_trn.csv", index = False )
clk_morn_tst.to_csv( loc_ftr + "\\clk_morn_tst.csv", index = False )

del clk_trn['clk_4_10']
del clk_tst['clk_4_10']